In [ ]:
# https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/

# Building powerful image classification models using very little data

- 작은 데이터 셋으로 강력한 이미지 분류 모델 설계하기
- 수백에서 수천 장 정도의 작은 데이터 셋을 가지고도 강한 성능을 낼수있는 모델을 만들어 볼것 


## 세가지 방법을 이용해 다룰것 

1. 작은 네트워크를 처음부터 학습(앞으로 사용할 방법의 평가기준)
2. 기존 네트워크의 병목적인 특징을 사용 
3. 기존 네트워크의 상단부 레이어 fine-tuning


## 케라스의 기능 

- **ImageDataGenerator**
    - 실시간 이미지 증가 (augmentation)
- **flow**
    - ImageDataGenerator 디버깅
- **fit_generator**
    - ImageDataGenerator를 이용한 케라스 모델 학습
- **Sequential**
    - 케라스 내 모델 설계 인터페이스
- **keras.applications**
    - 케라스에서 기존 이미지 분류 네트워크 불러오기
- 레이어 동결 (freezing)을 이용한 fine-tuning


# 1. 시작하기 : 학습 이미지 2,000장 

시작하기전 머신을 위해서 최상의 학습 환경을 조성 

- 케라스, SciPy, PIL을 설치해주세요. 
    - 만약 NVIDIA GPU를 사용하실 분이라면 cuDNN을 설치해주세요. 물론, 저희는 적은 양의 데이터만 다룰 것이기 때문에 GPU는 꼭 필요하지 않습니다.
    - 이미지를 학습, 테스트 세트로 나누어 아래와 같이 준비 
        - 확장자는 .png 혹은 .jpg를 권함 
        
> 만약 지피유 텐서플로우를 사용 할 경우 conda를 이용해서 설치 


```bash

# 작업트리

data/
	test/
		dogs/
			dog0001.jpg
			dog0002.jpg
			...
		cats/
			cat0001.jpg
			cat0002.jpg
			...
	train/
		...

```

고양이와 강아지 이외의 이미지 데이터를 찾으신다면 Flickr API를 활용해보는 것을 추천

태그기반 검색을 사용해서 라이센스 제약이 없는 이미지를 쉽게 찾을 수 있습니다. 

데이터는 캐글 데이터를 이용해 진행 

- 학습 데이터 
    - 고양이 : 1,000장
    - 강아지 : 1,000장 
    
- 검증 데이터
    - 고양이 : 400장
    - 강아지 : 400장


- 적은 데이터로 효과적인 모델을 학습할수 있는지 알아보기 위해서
    - 원본 데이터 25,000장 중에서 일부만 가져와 진행할것 

보통은 1,000장의 사진을 가지고 복잡한 이미지 분류 모델을 학습시키는 것은 정말 어렵다. 하지만 실제 상황에서 자주 접하게 될 법한 문제이다. 그렇기 때문에 적은 데이터로 최대의 성능을 내는것이 데이터 과학자의 핵심 자질이다. 


고양이 대 강아지 대회를 캐글에서 처음 진행했을 때 정확도 60%조차 달성하기 어려울 것이라고 전문가들은 단언했다. 하지만 이제는 80% 정확도 정도 도달할수있다. 


# 2. 데이터가 적은 문제에서 딥러닝 적용하기 

데이터가 정말 많을 때 비로서 딥러닝을 해볼수있다 ! 
컴퓨터가 데이터의 특성을 파악하기위해서는 학습할 수 있는 데이터가 많아야 한다. 
특히 이미지처럼 차원이 높은 데이터, 즉 복잡한 데이터는 더더욱 그렇다. 

하지만 대표적인 딥러닝 모델인 CNN은 바로 이런 문제를 해결하기 위해 설계된 모델이며
학습한 데이터가 적은 경우라도 별도의 데이터 조작없이 적은 데이터를 가지고도 간단한 CNN을 처음부터 학습시켜보면 괜찮은 성능이 나오는것을 확인할수 있다.


## 2.1 데이터 전처리와 Augmentation

모델이 적은 이미지에 최대한 많은 정보를 뽑아내서 학습할수 있도록 우선 이미지를 Augment 시켜 보갰다. 

이미지를 사용할 때 마다 임의로 변경을 가함으로써 마치 훨씬 더 많은 이미지를 보고 공부하는 것과 같은 학습 효과를 낸다. 
이를 통해서 과적합(overfitting), 즉, 모델이 학습 데이터에만 맞춰지는 것을 방지하고, 새로운 이미지도 잘 분류 할 수있게 한다. 


전처리 과정을 돕기 위해 케라스는 ImageDataGenerator 클래스를 제공한다. 

### ImageDataGenerator 하는 일

- 학습 도중에 이미지에 임의 변형 및 정규화 적용 
- 변형된 이미지를 배치 단위로 불러올 수 있는 generator 생성
    - generator를 생성할 때  flow(data, labels), flow_from_directory(directory) 두가지 함수를 사용한다. 
    - fit_generator, evaluate_generator 함수를 이용하여 generator로 이미지를 불러와서 모델을 학습한다. 

In [10]:
# # ImageDataGenerator
# from keras.preprocessing.image import ImageDataGenerator

# datagon = ImageDataGenerator(
#         rotation_range=40,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         rescale=1./255,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest')

```py

datagon = ImageDataGenerator(
        rotation_range=40,      # 이미지 회전 범위  
        width_shift_range=0.2,  # 수직(높이)으로 랜덤하게 평행 이동하는 범위
        height_shift_range=0.2, # 수평(넓이)으로 랜덤하게 평행 이동하는 범위
        rescale=1./255,         # 원본은 0-255의 RGB 계수로 구성 
                                # 입력 값은 모델을 효과적으로 학습하기엔 너무 높다. 
                                # 그래서 이것을 1/255로 스케일링을 하여 
                                # 0-1범위로 변환한다. 
                                # 다른 전처리 과정에 앞서서 가장 먼저 적용한다. 
        
        shear_range=0.2,        # 임의 전단 변환 (shearing transformation) 
        zoom_range=0.2,         # 임의 확대/축소 범위 
        horizontal_flip=True,   # True로 설정할 경우, 
                                # 50% 확률로 이미지를 수평으로 뒤집는다.
                                # 원본 이미지에 수평 비대칭성이 없을 때 효과적 
        fill_mode=`nearest`)    # 이미지를 회전, 이동하거나 축소할 때 
                                # 생기는 공간을 채우는 장식

```


- rotation_range: 이미지 회전 범위 (degrees)
- width_shift, height_shift: 그림을 수평 또는 수직으로 랜덤하게 평행 이동시키는 범위 (**원본 가로, 세로 길이에 대한 비율 값**)
- rescale: 원본 영상은 0-255의 RGB 계수로 구성되는데, 이 같은 입력값은 모델을 효과적으로 학습시키기에 너무 높습니다 (통상적인 learning rate를 사용할 경우). 그래서 이를 1/255로 스케일링하여 0-1 범위로 변환시켜줍니다. 이는 다른 전처리 과정에 앞서 가장 먼저 적용됩니다.
- shear_range: 임의 전단 변환 (shearing transformation) 범위
- zoom_range: 임의 확대/축소 범위
- horizontal_flip: True로 설정할 경우, 50% 확률로 이미지를 수평으로 뒤집습니다. 원본 이미지에 수평 비대칭성이 없을 때 효과적입니다. 즉, 뒤집어도 자연스러울 때 사용하면 좋습니다.
- fill_mode 이미지를 회전, 이동하거나 축소할 때 생기는 공간을 채우는 방식

## ImageDataGenerator 디버깅

Generator를 이용해서 학습하기전, 먼저 변형된 이미지에 이상한 점이 없는지 확인해야 한다.       
케라스는 이를 돕기 위해 flow라는 함수를 제공한다.      
여기서 reScale는 제외하고 진행한다. 255배 어두운 이미지는 아무래도 눈으로 확인하기 힘들기 때문이다. 

In [11]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [12]:
import os,glob

In [14]:
# os.getcwd()
# glob.glob('C://dogs-vs-cats/train/'+'*')

In [19]:
# 2 번 
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')


img = load_img(r'C:\\dogs-vs-cats\\train\\cat.1001.jpg') # PIL 이미지 
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.

# 반복문 

i = 0
for batch in datagen.flow(x, batch_size=1, save_to_dir='preview', save_prefix='cat', save_format='jpeg') :
    i += 1
    if i>20:
        break # 이미지 20장 생성

## 작은 CNN 밑바닥부터 학습하기 - 코드 40줄, 정확도 80%


이미지 분류 작업에 가장 적합한 모델은 CNN 이다. 
처음은 특별한 기법없이 소규모 CNN을 처음부터 학습할것이다.

데이터가 많지 않기 때문에 먼저 과적합 문제를 해결해야한다.
이미지를 효과적으로 분류하려면 대상의 핵심 특징에 집중해야한다. -> 기준이 뭐가될지를 잘정해야한다. 


Generator를 이용해서 이미지 augmentation을 수행하는 목적은 이런 과적합을 막는 데에 있다. 하지만 augmentation에도 한계가 있다. 이미지의 특성은 크게 달라지지 않기 때문

- 가장 중요한 것은 모델의 엔트로피 용량
    - 이말은 용량 확보를 위해서는 모델을 담을 수 있는 정보의 양을 줄이는 것
- 모델의 복잡도를 늘리려면 
    - 담을 수 있는 정보가 많아져서 이미지에 더 많은 특징을 파악가능
    - 하지만 그만큼 분류작업에 방해되는 특징도 찾게 될 것

- 모델의 복잡도를 줄이면 
    - 가장 중요한 몇 가지 특징에 집중해서 분류 대산을 정확하게 파악하고, 
    - 새로운 이미지도 정확히 분류해 낼 수 있게 된다.

### 엔트로피 용량을 조절하는 다양한 방법

- 대표적으로 모델에 관여하는 파아미터 개수를 조절하는 방법 -> 레이어의 크기 
- L1, L2 정규화 (regularization) 같은 가중치 정규화 기법
- 학습하면서 모든 가중치를 반복적으로 축소하는 방법
    - 결과적으로 핵심적인 특징에 대한 가중치만 남게 되는 효과적 

레이어 개수, 그리고 레이어당 필터 개수가 적은 소규모 CNN을 학습

- 학습 과정에서는 데이터 augmentation 및 드롭아웃 (dropout) 기법을 사용
- 드롭아웃은 과적합을 방지할 수 있는 또 다른 방법
- 하나의 레이어가 이전 레이어로부터 같은 입력을 두번 이상 받지 못하도록 하여 데이터 augmentation과 비슷한 효과 봄


### 예상 모델 

- Convolution 레이어 3개를 쌓아놓은 간단한 형태
- ReLU 활성화 함수를 사용하고, max-pooling을 적용
- CNN 기법을 처음 고안한 Yann LeCun이 1990년대에 제시한 아키텍처와 유사한 형태

In [32]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, MaxPooling2D

In [36]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(3, 150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3))) # 32
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3))) # 64
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

ValueError: Negative dimension size caused by subtracting 2 from 1 for 'max_pooling2d_4/MaxPool' (op: 'MaxPool') with input shapes: [?,1,148,32].

모델 상단에는 두개의 fully-connected 레이어를 배치 하고    
마지막으로 시그모이드(sigmoid) 활성화 레이어를 배치      
**만약 분류하고자 하는 클래스가 3가지 이상이면 활성화 레이어(softmax)를 사용**       


손실함수는 `binary_crossentropy`를 사용 
- 이거 역시 클래스가 3개 이상이면 `categorical_crossentropy` 사용 

In [ ]:
# fully-connected  레이어 

model.add(Flatten())  # 이전 CNN 레이어에서 나온 3차원 배열은 1차원으로 뽑아줍니다
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

### 본격적 데이터 로드 

`flow_from_directory()`함수를 사용하면 이미지가 저장된 폴더를 기준으로 라벨 정보와 함께 이미지를 불러온다. 

In [ ]:
batch_size = 16

# 학습 이미지에 적용한 augmentation 인자를 지정한다.
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않는다. 
# 모델 성능을 평가할 때에는 이미지 원본을 사용한다..
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


# 이미지를 배치 단위로 불러와 줄 generator

# 학습
train_generator = train_datagen.flow_from_directory(
        'data/train',  # 타겟 디렉토리
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

# 검증
validation_generator = validation_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')


# 테스트 
test_generator = test_datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

### 본격적인 학습 시작 

- 각 에폭 (epoch) 당 GPU에서는 약 20\~30초, CPU에서는 300\~400초 정도 걸린다. 
- 너무 급하지 않다면 CPU에서도 충분히 학습을 돌릴 수 있는 시간


In [1]:
model.fit_generator(
        train_generator,
        steps_per_epoch=1000 // batch_size,
        validation_data=validation_generator,
        epochs=50)
model.save_weights('first_try.h5')  # 많은 시간을 들여 학습한 모델인 만큼, 학습 후에는 꼭 모델을 저장해줍시다.

NameError: name 'model' is not defined

CNN 규모가 작고, 드롭아웃 계수도 높았기 때문에 50 에폭만으로는 과적합이 일어나지 않았던 것으로 예상
볼만한 점은 검증 정확도가 변동이 심하다는 것
정확도 (accuracy)라는 통계 지표가 변동성이 큰 것이기도 하고, 검증 이미지가 500장밖에 되지 않기 때문에 그렇습니다. 변동성이 큰 만큼, 신뢰성이 조금 떨어지는 지표


저희 모델이 정말로 제대로 학습이 된 건지, 단지 평가에서 운이 좋았던 것인지 확실하지 않은 겁니다. 


이럴 때 사용하기 좋은 방법으로 k-fold cross-validation이 있습니다. 같은 데이터로 더욱 신뢰할 수 있는 평가 결과를 낼 수 있지만 그만큼 시간이 오래 걸리는 방법

### 병목 특징

더욱 정교한 접근 방식은 대규모 데이터셋에 이미 학습된 기존 네트워크 (convolutional neural network)를 활용하는 것
- 이미지를 파악하기에 유용한 특징을 추출하는법 이미 공부-? 언제 ?
- 앞쪽 레이어에서 이러한 특징을 추출(1차 추출)하고, 뒤쪽 레이어에서 그 특징(2차로)을 가지고 이미지를 분류
- 앞쪽 레이어만 쏙 빌려와서 높은 성능의 모델 통과 시켜 특징이 추출 
- 앞쪽 레이어를 통과시켜 추출된 특징



- 기존의 네트워크를 활용하는 방식을 transfer learning
    - 대규모 데이터 셋에 학습한 내용을 새로운 상황으로 옮겨온것


저희는 대규모 ImageNet 데이터셋에 미리 학습된 VGG16 네트워크를 사용할 것 ->  ImageNet에 없는 클래스를 분류하는 데에도 병목 특징을 사용하여 뛰어난 모델을 만들 수있다.


VGG16에서 convolution 레이어로 이루어진 앞단만 가져온다.      
이미지 데이터를 이 부분 모델에 넣고 돌려서 마지막 레이어에서 출력되는 병목 특징을 NumPy 배열에 담는다.      
병목 특징은 모두 수치적인 값을 담는다. -> 병목 데이터를 가지고 학습한다.
- 병목 데이터는 학습, 검증, 테스트셋으로 분리해서 관리


동결 (freezing): 학습시키고자 하지 않은 레이어는 가중치를 동결시켜서 연산량을 줄일 수 있다.      
한 이미지당 (이미지를 무작위로 변형하여) bottleneck을 여러 개 생성하면 augmentation의 효과를 가진다.

In [ ]:
batch_size = 16

generator = datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,   # 라벨은 불러오지 않는다.
        shuffle=False)     # 출력되는 병목 특징이 어디서 왔는지 알 수 있도록 
                            # 입력 데이터의 순서를 유지
                            #(사전순으로 cat 1000장, dog 1000장 순서로 입력이 들어온다).

# 이미지를 모델에 입력시켜 결과를 가져온다.
# 본래 어떤 예측 결과가 출력되어야 하지만 
# 모델의 일부만 가져왔기 때문에 병목 특징이 출력
bottleneck_features_train = model.predict_generator(generator, 2000)
# 출력된 병목 데이터를 저장합니다.
np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_features_train)

generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model.predict_generator(generator, 800)
np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open('bottleneck_features_train.npy'))
# 앞서 언급한 바와 같이 병목 특징은 순서대로 추출되기 때문에 라벨 데이터는 아래와 같이 손쉽게 생성할 수 있습니다.
train_labels = np.array([0] * 1000 + [1] * 1000)

validation_data = np.load(open('bottleneck_features_validation.npy'))
validation_labels = np.array([0] * 400 + [1] * 400)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=50,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(`bottleneck_fc_model.h5`)